# Интеллектуальный анализ данных – весна 2022

# Домашнее задание 7: Деревья. Случайный лес

Правила:

- Домашнее задание оценивается в 10 баллов (+1 бонусный балл).


- Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.


- Можно использовать любые свободные источники с обязательным указанием ссылки на них.


- Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.

<!-- ![](meme.jpg) -->
<img src="meme.jpg" alt="Drawing" style="width: 700px;"/>

## Часть 1: Основы построения решающие дерева (1.5 балла)

В этой части все расчёты необходимо реализовывать в виде запрограммированных формул, например, на `numpy`. **Нельзя использовать готовые реализации**. Например, если в задании требуется рассчитать энтропию, то требуется в каком-то виде релизовать расчёт по формуле, но нельзя использовать готовую реализацию `some_module.entropy()`.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Задание 1.1 (0.5 балла)** Пусть известно, что в вершину решающего дерева попали 10 объектов, 8 из которых имеют метку класса $k_1$, а 2 имеют метку класса $k_2$. Рассчитайте энтропию такого распределения классов (с натуральным логарифмом). Ответ округлите до двух знаков после запятой.

In [2]:
import math

In [6]:
ans = 0.8 * math.log(0.8) + 0.2 * math.log(0.2)
print(round(ans, 2))

-0.5


**Задание 1.2 (0.5 балла)** Пусть дополнительно известно, что вершина из предыдущего задания не является листовой и возможно такое разбиение, что в левое поддерево попадут все объекты класса $k_1$, а в правое - класса $k_2$. Посчитайте критерий информативности:

$$
Q(R_m, j, t) = H(R_m) - \frac{|R_\ell|}{|R_m|}H(R_\ell) - \frac{|R_r|}{|R_m|}H(R_r),
$$

где $R_m$ - множество объектов в разбиваемой вершине, $j$ - номер признака, по которому происходит разбиение, $t$ - порог разбиения, $R_\ell$ - множество объектов в левом поддереве, $R_r$ - множество объектов в правом поддереве.

Теперь в качестве $H(R)$ будем использовать индекс Джини:

$$
H(R) = \sum_{k=1}^J p_k(1-p_k),
$$
где $J$ – общее количество классов (в нашем случае, $J = 2$).

Ответ округлите до двух знаков после запятой.

In [8]:
round((0.8 * 0.2) * 2, 2)

0.32

**Задание 1.3 (0.5 балла)** Пусть при построении дерева образовалась листовая вершина с 10 объектами, значения целевой переменной для которых следующие: [1, 10, 5, 18, 100, 30, 50, 61, 84, 47] (решается задача регрессии). Чему будут равны предсказания модели для этих объектов?

In [11]:
np.mean([1, 10, 5, 18, 100, 30, 50, 61, 84, 47])

40.6

## Часть 2: Решающие деревья (4.5 балла)

В этой части мы напишем и протестируем собственную реализацию решающего дерева.

In [12]:
from collections import Counter
from typing import Dict, List, Tuple, Union

**Задание 2.1 (1.5 балла)** Реализуйте функцию `find_best_split()`, которая должна находить оптимальное разбиение подмножества обучающей выборки в соответствии с информационным критерием из **Задания 1.2**. В качестве меры хаотичности $H(R)$ для задачи регрессии испольуйте дисперсию подвыборки, а для задачи классификации – критерий Джини (определён в том же задании).

Для категориальных признаков применяется наивный алгоритм разбиения: мы пытаемся найти одно значение, разбиение по которому сильнее всего увеличит критерий информативности. Иными словами, объекты с конкретным значением признака отправляем в левое поддерево, остальные - в правое. Обратите внимание, что это далеко не оптимальные способ учёта категориальных признаков. Например, можно было бы на каждое значение категориального признака создавать отдельное поддерево или использовать более сложные подходы. Подробнее об этом можно прочитать в конспектах [лекций](https://github.com/esokolov/ml-course-hse/blob/master/2019-fall/lecture-notes/lecture07-trees.pdf) по машинному обучению на ПМИ (раздел «Учёт категориальных признаков»).

В качестве подсказок реализации можете пользоваться кодом из бонусной части семинара по решающим деревьям.

**Бонус:** Разрешается делать цикл для перебора порогов, но возможна имплементация без него. За имплементацию без цикла – **бонус 1 балл**.

In [212]:
def H(R):
    p = np.bincount(R) / 6
    p = p[p != 0]
    anti_p = 1 - p
    return sum(p * anti_p)


        
    

In [327]:
def find_best_split(
    feature_vector: Union[np.ndarray, pd.DataFrame], 
    target_vector: Union[np.ndarray, pd.Series],
    task: str = "classification",
    feature_type: str = "real"
) -> Tuple[np.ndarray, np.ndarray, float, float]:
    """
    Указания:
    * Пороги, приводящие к попаданию в одно из поддеревьев пустого множества объектов, не рассматриваются.
    * В качестве порогов, нужно брать среднее двух сосдених (при сортировке) значений признака
    * Поведение функции в случае константного признака может быть любым.
    * При одинаковых приростах Джини или дисперсии нужно выбирать минимальный сплит.
    * За наличие в функции циклов балл будет снижен. Векторизуйте! :)

    :param feature_vector: вещественнозначный вектор значений признака
    :param target_vector: вектор классов объектов,  len(feature_vector) == len(target_vector)
    :param task: либо `classification`, либо `regression`
    :param feature_type: либо `real`, либо `categorical`
    
    :return thresholds: отсортированный по возрастанию вектор со всеми возможными порогами, по которым объекты можно
     разделить на две различные подвыборки, или поддерева
    :return ginis: вектор со значениями критерия Джини для каждого из порогов в thresholds len(ginis) == len(thresholds)
    :return threshold_best: оптимальный порог (число)
    :return gini_best: оптимальное значение критерия Джини (число)
    """
    # ᕕ(╭ರ╭ ͟ʖ╮•́)⊃¤=(————-
    
    
    a = np.array([feature_vector, target_vector])
    b = a[:, a[0].argsort()]
    features = b[0]
    targets = b[1]
    
    if task == 'regression':
        
        
        if feature_type == 'real':
            dict_var = {}
            dict_var2 = {}
            c = features[1:]
            c = np.append(c,features[-1])
            thresholds = ((features + c) / 2)
            thresholds = thresholds[:-1]
            for t in list(thresholds):
                variety = np.var(targets[list(np.where(features < t)[0])]) + np.var(targets[list(np.where(features > t)[0])])
                if variety not in dict_var:
                    dict_var[variety] = t
                dict_var2[t] = variety
            
            
            
        if feature_type == 'categorical':
            dict_var = {}
            dict_var2 = {}
            #b = a[:, a[0].argsort()]
            #features = b[0]
            #targets = b[1].astype(int)
            feature_values = np.unique(features)
            targets = targets.astype(int)
            for i in feature_values:
                variety = np.var(targets[list(np.where(features == i)[0])]) + np.var(targets[list(np.where(features != i)[0])])
                if variety not in dict_var:
                    dict_var[variety] = i
                dict_var2[i] = variety
    

  
        
    if task == 'classification':
        if feature_type == 'real':
            dict_var = {}
            dict_var2 = {}
            c = features[1:]
            c = np.append(c,features[-1])
            thresholds = ((features + c) / 2)
            thresholds = thresholds[:-1]
            for t in list(thresholds):
                R_l = list(targets[list(np.where(features < t)[0])])
                R_r = list(targets[list(np.where(features > t)[0])])
                variety = H(list(targets)) - (len(R_l) / len(list(targets))) * H(R_l) - (len(R_r) / len(list(targets))) * H(R_r)
                if variety not in dict_var:
                    dict_var[variety] = t
                dict_var2[t] = variety
                
        if feature_type == 'categorical':
            dict_var = {}
            dict_var2 = {}
            feature_values = np.unique(features)
            targets = targets

            for i in feature_values:

                R_l = list(targets[list(np.where(features == i)[0])])
                R_r = list(targets[list(np.where(features != i)[0])])
                variety = H(list(targets)) - (len(R_l) / len(list(targets))) * H(R_l) - (len(R_r) / len(list(targets))) * H(R_r)
                if variety not in dict_var:
                    dict_var[variety] = i
                dict_var2[i] = variety
    
            
    gini_best = min(dict_var.keys())
    threshold_best = dict_var[gini_best]

    thresholds = np.array(list(dict_var2.keys()))
    ginis = np.array(list(dict_var2.values()))
            
    return thresholds, ginis, threshold_best, gini_best
        
    
    

    pass

thresholds, variances, threshold_best, variance_best = find_best_split(
    X["CRIM"].to_numpy(), 
    y, 
    task="regression",
    feature_type="real"
)


b = a[:, a[0].argsort()]
features = b[0]
targets = b[1]
c = features[1:]
c = np.append(c,features[-1])
thresholds = ((features + c) / 2)
thresholds = thresholds[:-1]
all_t = np.ones((len(targets), len(targets))) * targets
up_tr = np.triu(all_t)[1:]
low_tr = np.tril(all_t)[:-1]
ginis_right = np.var(up_tr, axis = 1) 
ginis_left = np.var(low_tr, axis = 1) 
ginis = vars_right + vars_left
comb = np.array([thresholds, ginis])
comb_sort = comb[:, comb[1].argsort()]
best_comb = comb_sort[:, 0]
threshold_best = best_comb[0]
gini_best = best_comb[1]

In [312]:
X["CRIM"].to_numpy()

array([6.32000e-03, 2.73100e-02, 2.72900e-02, 3.23700e-02, 6.90500e-02,
       2.98500e-02, 8.82900e-02, 1.44550e-01, 2.11240e-01, 1.70040e-01,
       2.24890e-01, 1.17470e-01, 9.37800e-02, 6.29760e-01, 6.37960e-01,
       6.27390e-01, 1.05393e+00, 7.84200e-01, 8.02710e-01, 7.25800e-01,
       1.25179e+00, 8.52040e-01, 1.23247e+00, 9.88430e-01, 7.50260e-01,
       8.40540e-01, 6.71910e-01, 9.55770e-01, 7.72990e-01, 1.00245e+00,
       1.13081e+00, 1.35472e+00, 1.38799e+00, 1.15172e+00, 1.61282e+00,
       6.41700e-02, 9.74400e-02, 8.01400e-02, 1.75050e-01, 2.76300e-02,
       3.35900e-02, 1.27440e-01, 1.41500e-01, 1.59360e-01, 1.22690e-01,
       1.71420e-01, 1.88360e-01, 2.29270e-01, 2.53870e-01, 2.19770e-01,
       8.87300e-02, 4.33700e-02, 5.36000e-02, 4.98100e-02, 1.36000e-02,
       1.31100e-02, 2.05500e-02, 1.43200e-02, 1.54450e-01, 1.03280e-01,
       1.49320e-01, 1.71710e-01, 1.10270e-01, 1.26500e-01, 1.95100e-02,
       3.58400e-02, 4.37900e-02, 5.78900e-02, 1.35540e-01, 1.281

In [310]:
a = np.array([X["CRIM"].to_numpy(), y])

In [311]:
a

array([[6.3200e-03, 2.7310e-02, 2.7290e-02, ..., 6.0760e-02, 1.0959e-01,
        4.7410e-02],
       [2.4000e+01, 2.1600e+01, 3.4700e+01, ..., 2.3900e+01, 2.2000e+01,
        1.1900e+01]])

In [242]:
a = np.array([['2', '7', '2', '9', '1'], ['1', '2', '2', '1', '2']])
b = a[:, a[0].argsort()]
features = b[0]
targets = b[1]
b

array([['1', '2', '2', '7', '9'],
       ['2', '1', '2', '2', '1']], dtype='<U1')

In [243]:
features

array(['1', '2', '2', '7', '9'], dtype='<U1')

In [244]:
targets

array(['2', '1', '2', '2', '1'], dtype='<U1')

In [297]:
dict_var= {}
dict_var2 = {}

In [298]:
feature_values = np.unique(features)
targets = targets

for i in feature_values:

    R_l = list(targets[list(np.where(features == i)[0])])
    R_r = list(targets[list(np.where(features != i)[0])])
    variety = H(list(targets)) - (len(R_l) / len(list(targets))) * H(R_l) - (len(R_r) / len(list(targets))) * H(R_r)
    if variety not in dict_var:
        dict_var[variety] = i
    dict_var2[i] = variety
    

In [299]:
feature_values

array(['1', '2', '7', '9'], dtype='<U1')

In [300]:
dict_var

{0.0888888888888888: '1', 0.1444444444444444: '2', 0.1333333333333333: '9'}

In [301]:
dict_var2

{'1': 0.0888888888888888,
 '2': 0.1444444444444444,
 '7': 0.0888888888888888,
 '9': 0.1333333333333333}

In [277]:
gini_best = min(dict_var.keys())
threshold_best = dict_var[gini_best]

ginis = np.array(list(dict_var.keys()))
thresholds = np.array(list(dict_var.values()))

In [303]:
gini_best

0.0888888888888888

In [304]:
threshold_best 

'1'

In [305]:
ginis

array([0.08888889, 0.14444444, 0.08888889, 0.13333333])

In [306]:
thresholds

array(['1', '2', '7', '9'], dtype='<U1')

In [302]:
gini_best = min(dict_var.keys())
threshold_best = dict_var[gini_best]

thresholds = np.array(list(dict_var2.keys()))
ginis = np.array(list(dict_var2.values()))


In [234]:
b = a[:, a[0].argsort()]
b

array([['1', '2', '4', '7', '9'],
       ['2', '1', '2', '2', '1']], dtype='<U21')

In [191]:
features = b[0].astype(int)
targets = b[1]
c = features[1:]
c = np.append(c,features[-1])
thresholds = ((features + c) / 2)
thresholds = thresholds[:-1]


In [213]:
thresholds

array([1.5, 3. , 5.5, 8. ])

In [215]:
R_l = targets[list(np.where(features < 5.5)[0])]

In [216]:
R_l

array(['2', '1', '2'], dtype='<U21')

In [217]:
R_r = targets[list(np.where(features > t)[0])]

In [227]:
R_r = list(R_r)

In [229]:
H(R_r)

0.4444444444444445

In [219]:
H(R_r)

TypeError: Cannot cast array data from dtype('<U21') to dtype('int64') according to the rule 'safe'

In [231]:
for t in list(thresholds):
    
    R_l = list(targets[list(np.where(features < t)[0])])
    
    R_r = list(targets[list(np.where(features > t)[0])])
    
    variety = H(list(targets)) - (len(R_l) / len(list(targets))) * H(R_l) - (len(R_r) / len(list(targets))) * H(R_r)
    dict_var[variety] = t

In [232]:
dict_var

{49.55555555555555: 'a',
 25.6875: 8.0,
 42.6875: 1.5,
 53.666666666666664: 3.0,
 78.0: 5.5,
 0.0888888888888888: 1.5,
 0.1444444444444444: 3.0,
 0.14444444444444443: 5.5,
 0.1333333333333333: 8.0}

In [195]:
np.bincount(targets)

TypeError: Cannot cast array data from dtype('<U21') to dtype('int64') according to the rule 'safe'

In [223]:
x = ['1', '3', '10', '2', '1', '2']

In [224]:
p = np.bincount(x) / 6
p = p[p != 0]
p

array([0.33333333, 0.33333333, 0.16666667, 0.16666667])

In [208]:
anti_p = 1 - p
anti_p

array([0.66666667, 0.66666667, 0.83333333, 0.83333333])

In [210]:
sum(p * anti_p)

0.7222222222222223

In [199]:
sum(np.bincount(x))

6

In [ ]:
np.argmax(np.bincount(c))

In [187]:
dict_var

{49.55555555555555: 'a',
 25.6875: 8.0,
 42.6875: 1.5,
 53.666666666666664: 3.0,
 78.0: 5.5}

In [179]:
all_t = np.ones((len(targets), len(targets))) * targets
up_tr = np.triu(all_t)[1:]
low_tr = np.tril(all_t)[:-1]
ginis_right = np.var(up_tr, axis = 1) 
ginis_left = np.var(low_tr, axis = 1) 
ginis = vars_right + vars_left
comb = np.array([thresholds, ginis])
comb_sort = comb[:, comb[1].argsort()]
best_comb = comb_sort[:, 0]
threshold_best = best_comb[0]
gini_best = best_comb[1]

In [180]:
thresholds

array([1.5, 3. , 5.5, 8. ])

In [170]:
dict_var = {}
feature_values = np.unique(a[0])
targets = a[1].astype(int)
for i in feature_values:
    variety = np.var(targets[list(np.where(features == i)[0])]) + np.var(targets[list(np.where(features != i)[0])])
    dict_var[variety] = i
    

gini_best = min(dict_var.keys())
threshold_best = dict_var[gini_best]

ginis = np.array(list(dict_var.keys()))
thresholds = np.array(list(dict_var.values()))

In [171]:
ginis

array([49.55555556, 25.6875    , 42.6875    ])

In [172]:
gini_best 

25.6875

In [173]:
threshold_best

'b'

In [174]:
thresholds

array(['a', 'b', 'c'], dtype='<U1')

In [175]:
dict_var

{49.55555555555555: 'a', 25.6875: 'b', 42.6875: 'c'}

In [78]:
b = a[:, a[0].argsort()]
b

array([['a', 'a', 'a', 'b', 'c'],
       ['15', '6', '17', '1', '9']], dtype='<U21')

In [147]:
features = b[0]
targets = b[1].astype(int)

In [148]:
features

array(['a', 'a', 'a', 'b', 'c'], dtype='<U21')

In [153]:
list(np.where(features != 'a')[0])

[3, 4]

In [155]:
np.var(targets[list(np.where(features == 'a')[0])]) + np.var(targets[list(np.where(features != 'a')[0])])

38.888888888888886

In [117]:
targets

array([15,  6, 17,  1,  9])

In [140]:
i = np.where(features == 'a', b, 0)

In [141]:
i

array([['a', 'a', 'a', '0', '0'],
       ['15', '6', '17', '0', '0']], dtype='<U21')

In [133]:
j

array([0, 1, 2])

In [119]:
reversed_list_index = features[::-1].index('a')

last_index = len(features) - 1 - reversed_list_index
last_index

2

In [127]:
one_category = b[:,:last_index+1]
one_category

array([['a', 'a', 'a'],
       ['15', '6', '17']], dtype='<U21')

In [126]:
other_categories = b[:,last_index+1:]
other_categories

array([['b', 'c'],
       ['1', '9']], dtype='<U21')

In [129]:
np.var(one_category[1].astype(int)) + np.var(other_categories[1].astype(int))

38.888888888888886

In [112]:
features[features == 'a'].indexes()

AttributeError: 'numpy.ndarray' object has no attribute 'indexes'

In [105]:
all_t = np.ones((len(targets), len(targets))) * targets

In [106]:
all_t

array([[15.,  6., 17.,  1.,  9.],
       [15.,  6., 17.,  1.,  9.],
       [15.,  6., 17.,  1.,  9.],
       [15.,  6., 17.,  1.,  9.],
       [15.,  6., 17.,  1.,  9.]])

In [107]:
up_tr = np.triu(all_t)[1:]
up_tr

array([[ 0.,  6., 17.,  1.,  9.],
       [ 0.,  0., 17.,  1.,  9.],
       [ 0.,  0.,  0.,  1.,  9.],
       [ 0.,  0.,  0.,  0.,  9.]])

In [108]:
low_tr = np.tril(all_t)[:-1]
low_tr

array([[15.,  0.,  0.,  0.,  0.],
       [15.,  6.,  0.,  0.,  0.],
       [15.,  6., 17.,  0.,  0.],
       [15.,  6., 17.,  1.,  0.]])

In [88]:
uniqs, indexes = np.unique(b[0], return_inverse=True)

In [89]:
uniqs

array(['a', 'b', 'c'], dtype='<U21')

In [90]:
indexes

array([0, 0, 0, 1, 2])

In [91]:
b[1][indexes]

array(['15', '15', '15', '6', '17'], dtype='<U21')

In [ ]:
feature_values = np.unique(R_m[feature])
    
    for t in feature_values:
        Q_array.append(q_error(R_m, feature, t))
        
    opt_threshold = feature_values[np.argmin(Q_array)]
    
    return opt_threshold, Q_array

In [ ]:
def split_node(R_m: np.array, feature: str, t: float) -> Iterable[np.array]:
    """
    Split a fixed set of objects R_m given feature number and threshold t
    """
    mask = R_m[feature] < t
    return R_m[mask], R_m[~mask]


def q_error(R_m: np.array, feature: str, t: float) -> float:
    """
    Compute error criterion for given split parameters
    """
    R_left, R_right = split_node(R_m, feature, t)
    error = len(R_left) / len(R_m) * H(R_left) +  len(R_right)/ len(R_m) * H(R_right)
    return error

Эту функцию можно протестировать на датасете `Boston` из `sklearn`.

In [317]:
from sklearn.datasets import load_boston

boston = load_boston()
X = pd.DataFrame(data=boston["data"], columns=boston["feature_names"])
y = boston["target"]
X.head()

print(boston["DESCR"])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Выведите график зависимости значения критерия ошибки от порогового значения при разбиении вершины по признаку `CRIM`.

**Задание 2.2 (1 балл)** Разберитесь с написанным кодом решающего дерева, заполните пропуски в коде и реализуйте недостающий метод `_predict_node()`.

Построение дерева осуществляется согласно базовому жадному алгоритму, предложенному в лекции в разделе «Построение дерева».
- **Выбор лучшего разбиения** необходимо производить по критерию Джини.
- **Критерий останова:** все объекты в листе относятся к одному классу или ни по одному признаку нельзя разбить выборку.
- **Ответ в листе:** наиболее часто встречающийся класс в листе.

In [ ]:
class DecisionTree:
    
    def __init__(
        self, 
        feature_types: Union[List[str], np.ndarray], 
        max_depth: int = None, 
        min_samples_split: int = None, 
        min_samples_leaf: int = None,
        task: str = "classification"
    ) -> None:
        
        if np.any(list(map(lambda x: x != "real" and x != "categorical", feature_types))):
            raise ValueError("There is unknown feature type")

        # В этой переменной будем хранить узлы решающего дерева. Каждая вершина хранит в себе идентификатор того,
        # является ли она листовой. Листовые вершины хранят значение класса для предсказания, нелистовые - правого и
        # левого детей (поддеревья для продолжения процедуры предсказания)
        self._tree = {}
        
        # типы признаков (категориальные или числовые)
        self._feature_types = feature_types
        
        # гиперпараметры дерева
        self._max_depth = max_depth
        self._min_samples_split = min_samples_split
        self._min_samples_leaf = min_samples_leaf
        self.task = task

    def _fit_node(
        self, 
        sub_X: np.ndarray, 
        sub_y: np.ndarray, 
        node: dict
    ) -> None:
        
        # критерий останова
        if np.all(sub_y == sub_y[0]):
            node["type"] = "terminal"
            node["class"] = sub_y[0]
            return

        feature_best, threshold_best, gini_best, split = None, None, None, None
        for feature in range(sub_X.shape[1]):
            feature_type = self._feature_types[feature]
            categories_map = {}

            # подготавливаем признак для поиска оптимального порога
            if feature_type == "real":
                feature_vector = sub_X[:, feature]
            elif feature_type == "categorical":
                # здесь могла быть реализация более сложного подхода к обработке категориального признака
                feature_vector = sub_X[:, feature]

            # ищем оптимальный порог
            _, _, threshold, gini = find_best_split(feature_vector, sub_y, self.task, feature_type)
            
            if gini_best is None or gini > gini_best:
                feature_best = feature
                gini_best = gini

                # split - маска на объекты, которые должны попасть в левое поддерево
                if feature_type == "real":
                    threshold_best = threshold
                    split = # ᕕ(╭ರ╭ ͟ʖ╮•́)⊃¤=(————-
                elif feature_type == "categorical":
                    # в данной реализации это просто значение категории
                    threshold_best = threshold
                    split = # ᕕ(╭ರ╭ ͟ʖ╮•́)⊃¤=(————-
                else:
                    raise ValueError

        # записываем полученные сплиты в атрибуты класса
        if feature_best is None:
            node["type"] = "terminal"
            node["class"] = Counter(sub_y).most_common(1)[0][0]
            return

        node["type"] = "nonterminal"

        node["feature_split"] = feature_best
        if self._feature_types[feature_best] == "real":
            node["threshold"] = threshold_best
        elif self._feature_types[feature_best] == "categorical":
            node["category_split"] = threshold_best
        else:
            raise ValueError
            
        node["left_child"], node["right_child"] = {}, {}
        self._fit_node(sub_X[split], sub_y[split], node["left_child"])
        self._fit_node(sub_X[np.logical_not(split)], sub_y[np.logical_not(split)], node["right_child"])

    def _predict_node(self, x: np.ndarray, node: dict) -> int:
        """
        Предсказание начинается с корневой вершины дерева и рекурсивно идёт в левое или правое поддерево в зависимости от значения
        предиката на объекте. Листовая вершина возвращает предсказание.
        :param x: np.array, элемент выборки
        :param node: dict, вершина дерева
        """
        # ᕕ(╭ರ╭ ͟ʖ╮•́)⊃¤=(————-
        pass

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        self._fit_node(X, y, self._tree)

    def predict(self, X: np.ndarray) -> np.ndarray:
        predicted = []
        for x in X:
            predicted.append(self._predict_node(x, self._tree))
            
        return np.array(predicted)

**Задание 2.3 (1 балл)** Загрузите таблицу [students.csv](https://drive.google.com/file/d/0B2zoFVYw1rN3a0d0Zm43TzQ4aUU/view?usp=sharing) (это немного преобразованный датасет [User Knowledge](https://archive.ics.uci.edu/ml/datasets/User+Knowledge+Modeling)). В ней признаки объекта записаны в первых пяти столбцах, а в последнем записана целевая переменная (класс: 0 или 1). Постройте на одном изображении пять кривых "порог — значение критерия Джини" для всех пяти признаков. Отдельно визуализируйте диаграммы рассеяния "значение признака — класс" для всех пяти признаков.

Исходя из кривых значений критерия Джини, по какому признаку нужно производить деление выборки на два поддерева? Согласуется ли этот результат с визуальной оценкой диаграмм рассеяиния? Как бы охарактеризовали вид кривой для "хороших" признаков, по которым выборка делится почти идеально? Чем отличаются кривые для признаков, по которым деление практически невозможно?

**Задание 2.4 (1 балл)** Протестируйте свое решающее дерево на датасете [mushrooms](https://archive.ics.uci.edu/ml/datasets/Mushroom). 

1. Скачайте таблицу `agaricus-lepiota.data` (из [Data Folder](https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/)), 
2. Считайте таблицу при помощи `pandas`,
3. Примените к каждому столбцу `LabelEncoder` (из `sklearn`), чтобы преобразовать строковые имена категорий в натуральные числа. 

Первый столбец — это целевая переменная (e — edible, p — poisonous) Мы будем измерять качество с помощью accuracy, так что нам не очень важно, что будет классом 1, а что — классом 0. Обучите решающее дерево на половине случайно выбранных объектов (признаки в датасете категориальные) и сделайте предсказания для оставшейся половины. Вычислите accuracy.

## Часть 3: Бэггинг и случайный лес (4 балла)

В данной части мы будем работать [с задачей предсказания диабета у пациента](https://www.kaggle.com/uciml/pima-indians-diabetes-database/data). Посмотрим на работу бэггинга над решающими деревьями и случайного леса, сравним их работу.

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
data = pd.read_csv('diabetes.csv')
print(f"Dataset shape: {data.shape}")
data.head()

Посмотрим на распределение целевой переменной

In [ ]:
data['Outcome'].hist()
plt.show()

**Задание 3.1 (0.5 балла)** Разделите данные на признаки и целевую переменную. Разбейте датасет на обучающую и тестовую части в отношении 7:3. Затем разделите обучающую выборку на обучающую-обучающую и обучающую-валидационную в соотношении 7:3 (то есть в итоге должно получиться три выборки: обучающая-обучающая (0.49 от исходного датасета), обучающая-валидационная (0.21 от исходного датасета) и тестовая (0.3 от исходного датасета).

**Задание 3.2 (1 балл)** На обучающей-валидационной выборке подберите оптимальные значения гиперпараметров `max_depth` и `min_samples_leaf` для `DecisionTreeClassifier`. Для этого:
1. Создайте списки с возможными значениями для перебора.
2. Для каждой пары значений обучите дерево на обучающей-обучающей выборке и определите качество на обучающей-валидационной выборке. В качестве критерия будем использовать `f1-меру`.
3. Выберите ту пару значений, которая даёт наилучшее качество на обучающей-валидационной выборке. 


Обучите решающее дерево с подобранными гиперпараметрами на **полной обучающей** выборке. Оцените качество классификации на тестовой выборке по метрикам `accuracy`, `precision` и `recall`, `auc_roc`.

**Задание 3.3 (0.5 балла)** Обучите [`BaggingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) на 50 деревьях на **полной обучающей** выборке. Оцените качество классификации на тестовой выборке по тем же метрикам.

**Задание 3.4 (1 балл)** Выполните кросс-валидацию на полной обучающей выборке и подберите оптимальные значения гиперпараметров `max_depth` и `min_samples_split` для `Random Forest` с 50 деревьями. Для этого:

1. Создайте списки с возможными значениями для перебора.
2. Для каждой пары значений проведите кросс-валидацию на полной обучающей выборке. Количество разбиений выберите на ваш вкус. В качестве критерия будем использовать `f1-меру`. Усредните значение критерия по всем прогонам кросс-валидации. 
3. Выберите ту пару значений, которая даёт наилучшее среднее качество. 

Обучите случайный лес с подобранными гиперпараметрами на **полной обучающей** выборке. Оцените качество классификации по тем же метрикам. Какая из трёх построенных моделей показала себя лучше?

**Задание 3.5 (0.5 балла)** Постройте график зависимости AUC ROC на тестовой выборке от числа деревьев (`n_estimators`) для случайного леса, обучаемого на **полной обучающей** выборке. Какие выводы можно сделать?

**Задание 3.6 (0.5 балла)** Для лучшей модели случайного леса из **Задания 3.4** посчитайте важность признаков и постройте bar plot. Какой признак оказался самым важным для определения диабета?